In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import seaborn as sns
from pymongo import MongoClient
import geopandas as gpd
from cartoframes.viz import Map, Layer
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
load_dotenv()

True

In [2]:
client = MongoClient("localhost:27017")
db = client["Ironhack"] # I or i 
client_id= os.getenv("Client_Id")
client_s= os.getenv("Client_Secret")
client_key= os.getenv("Key")

In [3]:
initial_lat = 34.0218593
initial_long = -118.498265

In [4]:
LA_map = folium.Map(location = [initial_lat, initial_long], zoom_start=11)
LA_map

In [5]:
db.list_collection_names()

['bookstores',
 'districts',
 'nyrest',
 'nyrestaurants.',
 'restaurants',
 'companies',
 'distritos']

In [6]:
companies = db.get_collection('companies') # C or c

### function for creating the dataframe of all the info needed

In [7]:
# function for creating the dataframe of all the info needed

# lets drop some more, one of the most important requiremetns is having design companies nearby.
# We should remove all comnaies that are not design companies 

def Company_in_cali(c):
    filter_ = {'$and':[{'offices': {'$exists': 1}},{'offices.latitude':{'$type':1}},{'offices.longitude':{'$type': 1}}]}
    projection = {"_id":0,'name':1,'founded_year':1,'offices.country_code':1,"offices.state_code":1,'offices.city':1,'offices.address1':1,'offices.address2':1,'offices.latitude':1,'offices.longitude':1}
    sorting = {'offices.country_code':-1}
    lis = list(companies.find(filter_, projection).sort("offices.country_code", -1))
    df = pd.DataFrame(lis).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df.dropna(subset=["latitude"],inplace=True)
    df.dropna(subset=["founded_year"],inplace=True)
    df.dropna(subset=["address1"],inplace=True)
    df.dropna(subset=["address2"],inplace=True)
    df.dropna(subset=["city"],inplace=True)
    newcols = [element for element in df.columns if element != 'offices']
    df=df[newcols].reset_index(drop=True)
    df=df[df["country_code"]=="USA"]
    df=df[df['state_code'] == 'CA'].reset_index(drop=True)
    df = df.drop(columns="founded_year")
    return df


In [8]:
df= Company_in_cali(companies)
df

name                          address1     address2  \
0                        Yola                201 Mission Street   Suite 2250   
1                  Buongiorno                   485 Alberto Way    Suite 210   
2                      InMobi                       475 Brannan    Suite 420   
3                     LinQpay                                                  
4                     Sparkpr                   2 Bryant Street    Suite 100   
...                       ...                               ...          ...   
2396                 Cantimer                  3700 Haven Court                
2397  Celestial Semiconductor  2635 North 1st Street, Suite 150                
2398                 Purfresh                 1350 Willow Road,    Suite 102   
2399                MyChances                                                  
2400                  Intergy            11875 Dublin Boulevard  Suite A-201   

               city state_code country_code   latitude   longitude  
0     San Francisco         CA          USA  37.791115 -122.395475  
1         Los Gatos         CA          USA  37.229898 -121.971853  
2     San Francisco         CA          USA  37.446550 -122.170921  
3          Carlsbad         CA          USA  33.095680 -117.244034  
4     San Francisco         CA          USA  37.787394 -122.388442  
...             ...        ...          ...        ...         ...  
2396     Menlo Park         CA          USA  37.484616 -122.181753  
2397       San Jose         CA          USA  37.386417 -121.929170  
2398     Menlo Park         CA          USA  37.477008 -122.151599  
2399     Menlo Park         CA          USA  37.448491 -122.180281  
2400         Dublin         CA          USA  37.701940 -121.936698  

[2401 rows x 8 columns]

## Dataframe of all the employees and their count and salaries

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.

In [9]:
# Dataframe of all the employees and their count and salaries

employee_table = {'Title': ['Game designer', 'UX/UI', 'Front-end developer', 'Data engineer', 'Back-end developer',
                      'Account manager', 'Maintenance guy', 'Executive', 'CEO' ],
                'Salary': [115568, 104266, 87240, 112493, 80258, 58670, 40482, 52522, 148598],
                'Count': [20, 5, 10, 15, 5, 20, 1, 10,1]
                 } 
# We can sue this to weigh our decision, Dani did something like this,
# I think its good for reasoning why something is more important than another
# salaries are from glassdoor and are average salaries 

df_employee_table = pd.DataFrame(employee_table).explode("Title")
df_employee_table['total spending'] = df_employee_table.eval('Salary*Count')
df_employee_table.sort_values(by=['total spending'], ascending=False)

Title  Salary  Count  total spending
0        Game designer  115568     20         2311360
3        Data engineer  112493     15         1687395
5      Account manager   58670     20         1173400
2  Front-end developer   87240     10          872400
7            Executive   52522     10          525220
1                UX/UI  104266      5          521330
4   Back-end developer   80258      5          401290
8                  CEO  148598      1          148598
6      Maintenance guy   40482      1           40482

## Heatmapping function

In [10]:

def Heat_map_cali(df,radius):
    m = folium.Map(location = [34.0218593,-118.498265], zoom_start=10)
    HeatMap(data=df[["latitude","longitude"]],radius=radius).add_to(m)
    return m


In [11]:
Heat_map_cali(df,15)

## Foursquare api Work soo far


In [12]:
query = {"offices.city": "Los Angeles"}
projection = {"_id":0,"name":1,"offices.city":1, "country_code":1,
              "offices.latitude":1, "offices.longitude":1,"office.city":1}

list(companies.find(query, projection))[:5]

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]}]

In [13]:
len(list(companies.find(query, projection)))


219

In [14]:
LosAngeles = list(companies.find(query, projection))


In [15]:
LosAngeles

[{'name': 'Helio',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.057498,
    'longitude': -118.446596}]},
 {'name': 'Stickam',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.051409,
    'longitude': -118.254558}]},
 {'name': 'Ustream',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.392936,
    'longitude': -122.07948},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None},
   {'city': 'Budapest', 'latitude': None, 'longitude': None}]},
 {'name': 'TripUp',
  'offices': [{'city': 'Los Angeles',
    'latitude': 34.052187,
    'longitude': -118.243425}]},
 {'name': 'Eventful',
  'offices': [{'city': 'San Diego',
    'latitude': 32.872081,
    'longitude': -117.206292},
   {'city': 'Los Angeles', 'latitude': None, 'longitude': None}]},
 {'name': 'ScanScout',
  'offices': [{'city': 'Boston',
    'latitude': 42.340889,
    'longitude': -71.051931},
   {'city': 'New York', 'latitude': 40.739833, 'longitude': -73.993158},
   {'city': 'Los Angeles', 'la

In [16]:
LosAngeles[0]["offices"]

[{'city': 'Los Angeles', 'latitude': 34.057498, 'longitude': -118.446596}]

In [17]:
LosAngeles_new = []
for i in LosAngeles:
    for j in i['offices']:
        if j['city'] == 'Los Angeles':
            city = j['city']
            lat = j['latitude']
            lon = j['longitude']
            name = i['name']
            type_ = ({"typepoint": 
                      {"type": "Point", 
                         "coordinates": [j['latitude'], j['longitude']]}})
            
            LosAngeles_new.append({"name":name, "lat":lat, "long":lon, "type":type_})
LosAngeles_new

[{'name': 'Helio',
  'lat': 34.057498,
  'long': -118.446596,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.057498, -118.446596]}}},
 {'name': 'Stickam',
  'lat': 34.051409,
  'long': -118.254558,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.051409, -118.254558]}}},
 {'name': 'Ustream',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'TripUp',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.052187, -118.243425]}}},
 {'name': 'Eventful',
  'lat': None,
  'long': None,
  'type': {'typepoint': {'type': 'Point', 'coordinates': [None, None]}}},
 {'name': 'ScanScout',
  'lat': 34.050887,
  'long': -118.459995,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.050887, -118.459995]}}},
 {'name': 'WebbAlert',
  'lat': 34.052187,
  'long': -118.243425,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [34.0

In [18]:
LosAngeles[0]["name"]


'Helio'

In [19]:
LosAngeles[0]["offices"]


[{'city': 'Los Angeles', 'latitude': 34.057498, 'longitude': -118.446596}]

In [20]:
len(LosAngeles_new)

220

In [21]:
df_LosAngeles = pd.DataFrame(LosAngeles_new)
df_LosAngeles

name        lat        long  \
0                          Helio  34.057498 -118.446596   
1                        Stickam  34.051409 -118.254558   
2                        Ustream        NaN         NaN   
3                         TripUp  34.052187 -118.243425   
4                       Eventful        NaN         NaN   
..                           ...        ...         ...   
215         California 7 Studios        NaN         NaN   
216            Pipit Interactive  34.045499 -118.470360   
217  Top Rated Website Designers  33.993326 -118.398784   
218                 SOA Software        NaN         NaN   
219             IVR Technologies  34.050066 -118.253161   

                                                  type  
0    {'typepoint': {'type': 'Point', 'coordinates':...  
1    {'typepoint': {'type': 'Point', 'coordinates':...  
2    {'typepoint': {'type': 'Point', 'coordinates':...  
3    {'typepoint': {'type': 'Point', 'coordinates':...  
4    {'typepoint': {'type': 'Point', 'coordinates':...  
..                                                 ...  
215  {'typepoint': {'type': 'Point', 'coordinates':...  
216  {'typepoint': {'type': 'Point', 'coordinates':...  
217  {'typepoint': {'type': 'Point', 'coordinates':...  
218  {'typepoint': {'type': 'Point', 'coordinates':...  
219  {'typepoint': {'type': 'Point', 'coordinates':...  

[220 rows x 4 columns]

In [22]:
df_LosAngeles.isna().sum()

name     0
lat     52
long    52
type     0
dtype: int64

In [23]:
df_LosAngeles_copy = df_LosAngeles.copy()

In [24]:
df_LosAngeles_copy.dropna(subset=['lat', 'long'],inplace=True)

In [25]:
df_LosAngeles_copy.isna().sum()


name    0
lat     0
long    0
type    0
dtype: int64

In [26]:
len(df_LosAngeles_copy)

168

In [27]:
df_LosAngeles_copy

name        lat        long  \
0                          Helio  34.057498 -118.446596   
1                        Stickam  34.051409 -118.254558   
3                         TripUp  34.052187 -118.243425   
5                      ScanScout  34.050887 -118.459995   
6                      WebbAlert  34.052187 -118.243425   
..                           ...        ...         ...   
210             AECOM Technology  34.050796 -118.256861   
214                  Culture Jam  34.013715 -118.466060   
216            Pipit Interactive  34.045499 -118.470360   
217  Top Rated Website Designers  33.993326 -118.398784   
219             IVR Technologies  34.050066 -118.253161   

                                                  type  
0    {'typepoint': {'type': 'Point', 'coordinates':...  
1    {'typepoint': {'type': 'Point', 'coordinates':...  
3    {'typepoint': {'type': 'Point', 'coordinates':...  
5    {'typepoint': {'type': 'Point', 'coordinates':...  
6    {'typepoint': {'type': 'Point', 'coordinates':...  
..                                                 ...  
210  {'typepoint': {'type': 'Point', 'coordinates':...  
214  {'typepoint': {'type': 'Point', 'coordinates':...  
216  {'typepoint': {'type': 'Point', 'coordinates':...  
217  {'typepoint': {'type': 'Point', 'coordinates':...  
219  {'typepoint': {'type': 'Point', 'coordinates':...  

[168 rows x 4 columns]

In [28]:
LA_group = folium.FeatureGroup(name="Los Angeles")


In [29]:
HeatMap(data=df_LosAngeles_copy[["lat", "long"]], radius=15,
       gradient = {0.4: 'yellow', 0.65: 'orange', 1: 'red'}).add_to(LA_group)

In [30]:
LA_group.add_to(LA_map)


In [31]:
LA_map

In [32]:
SM_lat = 34.0195
SM_long = -118.4912

In [33]:
SM_map = folium.Map(location = [SM_lat, SM_long], zoom_start=10)
folium.Marker([SM_lat, SM_long]).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=500).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=1000).add_to(SM_map)
folium.Circle([SM_lat, SM_long], radius=3000).add_to(SM_map)

In [34]:
SM_map

In [35]:
SM_lat_updated = 34.03327701719273
SM_long_updated = -118.47431848484814


In [36]:
SM_map_updated = folium.Map(location = [SM_lat, SM_long], zoom_start=10)
folium.Marker([SM_lat_updated, SM_long_updated]).add_to(SM_map)
folium.Circle([SM_lat_updated, SM_long_updated], radius=500).add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=1000).add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=2000).add_to(SM_map_updated)
folium.Circle([SM_lat_updated, SM_long_updated], radius=3000).add_to(SM_map_updated)

In [37]:
SM_map_updated

In [38]:
def querySM (query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {
    "Accept": "application/json",
    "Authorization": f"{client_key}" 
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
    return new_list

In [39]:
daycare = querySM("daycare",[34.03327701719273, -118.47431848484814],3000)

UnboundLocalError: local variable 'new_list' referenced before assignment

In [ ]:
design_studio = querySM("Design_studio",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
airport = querySM("airport",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
bar = querySM("bar",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
Starbucks = querySM("starbucks",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
basketball_stadium = querySM("basketball_stadium",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
groomer = querySM("Pet Grooming Service",[34.03327701719273, -118.47431848484814],3000)

In [ ]:
vegan = querySM("vegan",[34.03327701719273, -118.47431848484814],3000)